# Use Deep Learning to Clone Driving Behavior

Overview
---
Train, validate and test a model using Keras to clone driving behavior. The model will output a steering angle to an autonomous vehicle.
Data will be collected by running a car manually in Udacity's Self-Driving Car simulator. The model performance will be tested by running the car in the simulator in autonomous mode.

#### The goals / steps of this project are the following:

* Use the simulator to collect data of good driving behavior
* Build, a convolution neural network in Keras that predicts steering angles from images
* Train and validate the model with a training and validation set
* Test that the model successfully drives around track one without leaving the road
* Summarize the results with a written report

Project Deliverables
---
* `model.py` contains the code for building the keras model based on the [Nvidia architecture](https://devblogs.nvidia.com/parallelforall/deep-learning-self-driving-cars/). 
* `model.h5` is the trained keras model needed to run the vehicle autonomously.
* `drive.py` is used to drive the vehicle autonomously in the simulator. It accepts an h5 file as a parameter.
* `video.mp4` is the video produced from the vehicle's camera view during the autonomous run.
* `video_rear.mp4` is the video of the same run looking at the vehicle from behind.

#### Importing libraries

In [ ]:
import os
import csv
from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle
import numpy as np
import cv2

#### Read collected data from the simulator
 
Image data and corresponding steering angle were collected by manually driving the vehicle in the simulator for 3 laps. The vehicle has 3 cameras to record the images - left, center, and right. The steering angles for the images from the left camera were adjusted by 0.1, while the angles for the images from the right camera were adjusted by -0.1 to consider different perspective view from each camera. This resulted in a total of 10,446 (3,482 x 3) images. 

 The following are the left, center, and right images recorded at the same location:

![Left](https://cloud.githubusercontent.com/assets/10526591/24032112/b7a5090e-0b29-11e7-8d0a-d54419d4c278.jpg)
![Center](https://cloud.githubusercontent.com/assets/10526591/24032113/b7c63c14-0b29-11e7-878f-663096e30985.jpg)
![Right](https://cloud.githubusercontent.com/assets/10526591/24032111/b7843300-0b29-11e7-9b89-6f4da4eaf539.jpg)


In [9]:
from io import StringIO
import requests
import json
import pandas as pd

# @hidden_cell
# This function accesses a file in your Object Storage. The definition contains your credentials.
# You might want to remove those credentials before you share your notebook.
def get_object_storage_file_with_credentials_83e0455de72a4323ae30b7f17b282631(container, filename):
    """This functions returns a StringIO object containing
    the file content from Bluemix Object Storage."""

    url1 = ''.join(['https://identity.open.softlayer.com', '/v3/auth/tokens'])
    data = {'auth': {'identity': {'methods': ['password'],
            'password': {'user': {'name': 'member_33c4989825a36840cc560705f473ac4e52e87422','domain': {'id': 'bf05c10c9a8f4077b8577c31fce9138b'},
            'password': 'g.I?Pse5Z2VDG7Z5'}}}}}
    headers1 = {'Content-Type': 'application/json'}
    resp1 = requests.post(url=url1, data=json.dumps(data), headers=headers1)
    resp1_body = resp1.json()
    for e1 in resp1_body['token']['catalog']:
        if(e1['type']=='object-store'):
            for e2 in e1['endpoints']:
                        if(e2['interface']=='public'and e2['region']=='dallas'):
                            url2 = ''.join([e2['url'],'/', container, '/', filename])
    s_subject_token = resp1.headers['x-subject-token']
    headers2 = {'X-Auth-Token': s_subject_token, 'accept': 'application/json'}
    resp2 = requests.get(url=url2, headers=headers2)
    return StringIO(resp2.text)

# Your data file was loaded into a StringIO object and you can process the data.
# Please read the documentation of pandas to learn more about your possibilities to load your data.
# pandas documentation: http://pandas.pydata.org/pandas-docs/stable/io.html
data_1 = get_object_storage_file_with_credentials_83e0455de72a4323ae30b7f17b282631('SelfDrivingCarDemo', 'data.zip')


In [ ]:
lines = []
with open('./data/driving_log.csv') as csvfile:
    reader = csv.reader(csvfile)
    for line in reader:
        lines.append(line)

In [ ]:
shuffle(lines)
train_data, validation_data = train_test_split(lines, test_size=0.2)

In [ ]:
images = []
angles = []
for line in train_data:
    # use left, center, right camera images
    angle_correction = 0.1
    for i in range(3):
        name = './data/IMG/' + line[i].split('\\')[-1]
        image = cv2.imread(name)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        images.append(image)
        angle = float(line[3])
        # adjust steering wheel angle for different camera views
    angles.append(angle)
    angles.append(angle + angle_correction)
    angles.append(angle - angle_correction)
print("done loading {} images".format(len(images)))
images_with_flipped = []
angles_with_flipped = []

# add flipped images and angles
for image, angle in zip(images, angles):
    images_with_flipped.append(image)
    angles_with_flipped.append(angle)
    flipped_image = np.fliplr(image)
    flipped_angle = -angle
    images_with_flipped.append(flipped_image)
    angles_with_flipped.append(flipped_angle)
print("done flipping and loading {} images".format(len(images_with_flipped)))

In [ ]:
# prepare validation set
val_images = []
val_angles = []
for line in validation_data:
    # use left, center, right camera images
    angle_correction = 0.1
    for i in range(3):
        name = './data/IMG/' + line[i].split('\\')[-1]
        image = cv2.imread(name)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        val_images.append(image)
        angle = float(line[3])
        # adjust steering wheel angle for different camera views
    val_angles.append(angle)
    val_angles.append(angle + angle_correction)
    val_angles.append(angle - angle_correction)
print("done loading {} validation images".format(len(val_images)))

X_train = np.array(images_with_flipped)
y_train = np.array(angles_with_flipped)
X_val = np.array(val_images)
y_val = np.array(val_angles)


Model Architecture
---
I based my model on the [Nvidia architecture](https://devblogs.nvidia.com/parallelforall/deep-learning-self-driving-cars/). It includes a normalization layer, 5 convolutional layers,1 dropout layer, and finally 3 fully connected layers. 
Through this model, the vehicle was able to drive autonomously around the track without leaving the road or diving into water and making smooth turns around corners.

In [ ]:
import keras
from keras.models import Sequential
from keras.layers import Flatten, Dense, Lambda, Dropout
from keras.layers.convolutional import Convolution2D, Cropping2D

model = Sequential()
model.add(Lambda(lambda x: x / 255.0 - 0.5, input_shape=(160, 320, 3)))
model.add(Cropping2D(cropping=((70, 25), (0, 0))))
model.add(Convolution2D(24, 5, 5, subsample=(2, 2), activation='relu'))
model.add(Convolution2D(36, 5, 5, subsample=(2, 2), activation='relu'))
model.add(Convolution2D(48, 5, 5, subsample=(2, 2), activation='relu'))
model.add(Convolution2D(64, 3, 3, activation='relu'))
model.add(Convolution2D(64, 3, 3, activation='relu'))
model.add(Dropout(0.8))
model.add(Flatten())
model.add(Dense(100))
model.add(Dense(50))
model.add(Dense(10))
model.add(Dense(1))

model.compile(optimizer='adam', loss='mse')
model.fit(X_train, y_train, validation_data = (X_val, y_val), nb_epoch=5)

In [ ]:
model.save('model.h5')

Results
---
View the **[video](https://www.youtube.com/watch?v=MB-ii0qzUmQ)** on Youtube

In [16]:
from IPython.display import HTML
HTML("""
<video width="640" height="480" controls>
  <source src="https://github.com/aruizga7/Self-Driving-Car-in-DSX/raw/master/5.%20Behavioral%20cloning/Self%20Driving%20Behavior%20Cloning.mp4">
</video>
""")

####  How to run the model in autonomous mode
Using the Udacity provided simulator and my drive.py file, the car can be driven autonomously around the track by executing 
```
python drive.py model.h5
```

####  Model Pipeline

The model.py file contains the code for training and saving the convolution neural network. The file shows the pipeline I used for training and validating the model, and it contains comments to explain how the code works.
